# Time Travel with Groq



## Features Demonstrated:
- Browsing state history
- Replaying from checkpoints
- Forking execution paths

## Installation

In [1]:
pip install -q langgraph langchain_groq langgraph_sdk langgraph-prebuilt

Note: you may need to restart the kernel to use updated packages.


## Setup API Key

In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

## Define Tools

In [3]:
from langchain_groq import ChatGroq

def multiply(a: int, b: int) -> int:
    """Multiply a and b.
    
    Args:
        a: first int
        b: second int
    """
    return a * b

def add(a: int, b: int) -> int:
    """Adds a and b.
    
    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a by b.
    
    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
# Using llama-3.3-70b-versatile which supports tool calling
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
llm_with_tools = llm.bind_tools(tools)

print("Tools defined successfully!")

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Tools defined successfully!


## Create the Graph

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)

# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

# Compile with memory
graph = builder.compile(checkpointer=MemorySaver())

print("Graph created successfully!")

Graph created successfully!


## Run the Agent

In [5]:
from langchain_core.messages import HumanMessage

# Input
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph
print("Running agent...\n")
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

Running agent...

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (p7cwh5gfr)
 Call ID: p7cwh5gfr
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


## Browse History

We can use `get_state` to look at the current state of our graph.

In [6]:
current_state = graph.get_state({'configurable': {'thread_id': '1'}})
print("Current State:")
print(f"Number of messages: {len(current_state.values['messages'])}")
print(f"Next steps: {current_state.next}")

Current State:
Number of messages: 4
Next steps: ()


Get all historical states:

In [7]:
all_states = [s for s in graph.get_state_history(thread)]
print(f"Total states in history: {len(all_states)}")

Total states in history: 5


Look at a specific state (the one with human input):

In [8]:
print("State at step -2:")
print(f"Messages: {all_states[-2].values['messages']}")
print(f"Next: {all_states[-2].next}")

State at step -2:
Messages: [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='0ead9494-349b-4089-b09c-7d8b04419f7b')]
Next: ('assistant',)


## Replaying

We can re-run our agent from any prior checkpoint.

In [9]:
to_replay = all_states[-2]
print("State to replay:")
print(f"Messages: {to_replay.values['messages']}")
print(f"Next node: {to_replay.next}")
print(f"Checkpoint ID: {to_replay.config['configurable']['checkpoint_id']}")

State to replay:
Messages: [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='0ead9494-349b-4089-b09c-7d8b04419f7b')]
Next node: ('assistant',)
Checkpoint ID: 1f0b19aa-b571-663a-8000-594d9e7baddc


Replay from the checkpoint:

In [10]:
print("Replaying from checkpoint...\n")
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    event['messages'][-1].pretty_print()

Replaying from checkpoint...

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (a1m1zwzff)
 Call ID: a1m1zwzff
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


## Forking

We can fork from a checkpoint by modifying the state and creating a new execution path.

In [11]:
to_fork = all_states[-2]
print("Original message:")
print(to_fork.values["messages"])

Original message:
[HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='0ead9494-349b-4089-b09c-7d8b04419f7b')]


Update the state with a new message (keeping the same ID to overwrite):

In [12]:
fork_config = graph.update_state(
    to_fork.config,
    {"messages": [HumanMessage(
        content='Multiply 5 and 3',
        id=to_fork.values["messages"][0].id
    )]}
)
print("Forked config created:")
print(f"New checkpoint ID: {fork_config['configurable']['checkpoint_id']}")

Forked config created:
New checkpoint ID: 1f0b19aa-be49-6334-8001-c624f1486499


Verify the state was updated:

In [13]:
all_states = [state for state in graph.get_state_history(thread)]
print("Updated first message:")
print(all_states[0].values["messages"])

Updated first message:
[HumanMessage(content='Multiply 5 and 3', additional_kwargs={}, response_metadata={}, id='0ead9494-349b-4089-b09c-7d8b04419f7b'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'p7cwh5gfr', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 373, 'total_tokens': 392, 'completion_time': 0.044432295, 'prompt_time': 0.035572388, 'queue_time': 0.049883892, 'total_time': 0.080004683}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--83141de2-4b4e-413e-9ef2-f400bae5ef1a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'p7cwh5gfr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 373, 'output_tokens': 19, 'total_tokens': 392})]


Run the forked version:

In [14]:
print("Running forked version...\n")
for event in graph.stream(None, fork_config, stream_mode="values"):
    event['messages'][-1].pretty_print()

Running forked version...

================================== Ai Message ==================================
Tool Calls:
  multiply (p7cwh5gfr)
 Call ID: p7cwh5gfr
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================
Tool Calls:
  multiply (3jpbd9g5s)
 Call ID: 3jpbd9g5s
  Args:
    a: 5
    b: 3
================================= Tool Message =================================
Name: multiply

15
================================== Ai Message ==================================

The result of multiplying 5 and 3 is 15.


Check the final state:

In [15]:
final_state = graph.get_state({'configurable': {'thread_id': '1'}})
print("Final state:")
print(f"Number of messages: {len(final_state.values['messages'])}")
print(f"Last message: {final_state.values['messages'][-1].content}")
print(f"Next steps: {final_state.next}")

Final state:
Number of messages: 6
Last message: The result of multiplying 5 and 3 is 15.
Next steps: ()
